***Vorlesung 'Syntax natürlicher Sprachen'***

--- 
# Vorlesung 9:Verarbeitungsprobleme CFG-Parsingalgorithmen


In [1]:
import nltk

---

## 1. Verbarbeitung Linksrekursion: nicht mit RecursiveDescent-Parser möglich

- linksrekursive Regeln: `NP -> NP PP`, `VP -> VP PP`

In [2]:
grammar = nltk.CFG.fromstring("""
## Syntaktische Regeln:
    S -> NP VP
    PP -> P NP
    NP -> Det N 
#rekursive Regel:
    NP -> NP PP
    NP -> Pron
    VP -> V NP
#rekursive Regel:    
    VP -> VP PP
##Lexikalische Regeln:
    Pron -> 'I'      
    Det -> 'an' | 'my'
    N -> 'elephant' | 'pajamas'
    V -> 'shot'
    P -> 'in'
""")

In [3]:
sent = 'I shot an elephant'.split()
print(sent)

['I', 'shot', 'an', 'elephant']


---
### 1.1 Recursive-Descent-Parser (einfacher top-down-Parser)
- https://www.nltk.org/book/ch08.html#recursive-descent-parsing

##### *Endlosschleife bei linksrekursiver Regel!*

In [4]:
#parser = nltk.RecursiveDescentParser(grammar,trace=2)
#for tree in parser.parse(sent):
#    print(tree)
#    tree.pretty_print(unicodelines=True)

```
Parsing 'I shot an elephant'
    [ * S ]
  E [ * NP VP ]
  E [ * Det N VP ]
  E [ * 'an' N VP ]
  E [ * 'my' N VP ]
  E [ * NP PP VP ]
  E [ * Det N PP VP ]
  E [ * 'an' N PP VP ]
  E [ * 'my' N PP VP ]
  E [ * NP PP PP VP ]
  E [ * Det N PP PP VP ]
  E [ * 'an' N PP PP VP ]
  E [ * 'my' N PP PP VP ]
  E [ * NP PP PP PP VP ]
  E [ * Det N PP PP PP VP ]
  E [ * 'an' N PP PP PP VP ]
  E [ * 'my' N PP PP PP VP ]
  E [ * NP PP PP PP PP VP ]
  E [ * Det N PP PP PP PP VP ]
  E [ * 'an' N PP PP PP PP VP ]
  E [ * 'my' N PP PP PP PP VP ]
  E [ * NP PP PP PP PP PP VP ]
  E [ * Det N PP PP PP PP PP VP ]
  E [ * 'an' N PP PP PP PP PP VP ]
  E [ * 'my' N PP PP PP PP PP VP ]
  E [ * NP PP PP PP PP PP PP VP ]
  E [ * Det N PP PP PP PP PP PP VP ]
  E [ * 'an' N PP PP PP PP PP PP VP ]
  E [ * 'my' N PP PP PP PP PP PP VP ]
  ...
```

---
### 1.2 Parsing möglich z.B. mit Shift-Reduce-Parser (bottom-up)
- https://www.nltk.org/book/ch08.html#shift-reduce-parsing

In [5]:
parser = nltk.ShiftReduceParser(grammar,trace=2)
for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print()

Parsing 'I shot an elephant'
    [ * I shot an elephant]
  S [ 'I' * shot an elephant]
  R [ Pron * shot an elephant]
  R [ NP * shot an elephant]
  S [ NP 'shot' * an elephant]
  R [ NP V * an elephant]
  S [ NP V 'an' * elephant]
  R [ NP V Det * elephant]
  S [ NP V Det 'elephant' * ]
  R [ NP V Det N * ]
  R [ NP V NP * ]
  R [ NP VP * ]
  R [ S * ]
(S (NP (Pron I)) (VP (V shot) (NP (Det an) (N elephant))))
           S                  
  _________|___                
 |             VP             
 |     ________|___            
 NP   |            NP         
 |    |         ___|_____      
Pron  V       Det        N    
 |    |        |         |     
 I   shot      an     elephant



---

### 1.3 ebenso möglich mit Chart-Parser (Parser mit dynamischer Programmierung)

https://www.nltk.org/book/ch08-extras.html#chart-parsing

#### 2 Strategien im NLTK:
- `BU_STRATEGY` (bottom-up)
- `TD_STRATEGY` (top-down)

> NLTK defines a simple yet flexible chart parser, `ChartParser`. A new chart parser is constructed from a grammar and a strategy. The strategy is applied until no new edges are added to the chart. NLTK defines two ready-made strategies: `TD_STRATEGY`, a basic top-down strategy; and `BU_STRATEGY`, a basic bottom-up strategy. When constructing a chart parser, you can use either of these strategies, or create your own. [...] `parser = nltk.ChartParser(groucho_grammar, nltk.parse.BU_STRATEGY)`(https://www.nltk.org/book/ch08-extras.html#chart-parsing)


- s. https://www.nltk.org/howto/parse.html

In [6]:
parser = nltk.ChartParser(grammar, trace=1) #Default Strategie = BottomUpChartParser
for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print()

|.    I    .   shot  .    an   . elephant.|
|[---------]         .         .         .| [0:1] 'I'
|.         [---------]         .         .| [1:2] 'shot'
|.         .         [---------]         .| [2:3] 'an'
|.         .         .         [---------]| [3:4] 'elephant'
|[---------]         .         .         .| [0:1] Pron -> 'I' *
|[---------]         .         .         .| [0:1] NP -> Pron *
|[--------->         .         .         .| [0:1] S  -> NP * VP
|[--------->         .         .         .| [0:1] NP -> NP * PP
|.         [---------]         .         .| [1:2] V  -> 'shot' *
|.         [--------->         .         .| [1:2] VP -> V * NP
|.         .         [---------]         .| [2:3] Det -> 'an' *
|.         .         [--------->         .| [2:3] NP -> Det * N
|.         .         .         [---------]| [3:4] N  -> 'elephant' *
|.         .         [-------------------]| [2:4] NP -> Det N *
|.         .         [------------------->| [2:4] S  -> NP * VP
|.         .         

In [7]:
parser = nltk.parse.EarleyChartParser(grammar, trace=1)
#parser = nltk.parse.TopDownChartParser(grammar, trace=1)
for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print()

|.    I    .   shot  .    an   . elephant.|
|[---------]         .         .         .| [0:1] 'I'
|.         [---------]         .         .| [1:2] 'shot'
|.         .         [---------]         .| [2:3] 'an'
|.         .         .         [---------]| [3:4] 'elephant'
|>         .         .         .         .| [0:0] S  -> * NP VP
|>         .         .         .         .| [0:0] NP -> * Det N
|>         .         .         .         .| [0:0] NP -> * NP PP
|>         .         .         .         .| [0:0] NP -> * Pron
|>         .         .         .         .| [0:0] Pron -> * 'I'
|[---------]         .         .         .| [0:1] Pron -> 'I' *
|[---------]         .         .         .| [0:1] NP -> Pron *
|[--------->         .         .         .| [0:1] S  -> NP * VP
|[--------->         .         .         .| [0:1] NP -> NP * PP
|.         >         .         .         .| [1:1] PP -> * P NP
|.         >         .         .         .| [1:1] VP -> * V NP
|.         >         .       

---
## 2. Verarbeitungsproblem Backtracking

- z.B. notwendig bei Sätzen mit temporaler Ambiguität

##### nicht in allen Parsern implementiert!

In [8]:
sent = 'the old man the boat'.split()

grammar = nltk.CFG.fromstring("""
## Syntaktische Regeln:
    S -> NP VP
    NP -> Det Adj N
    NP -> Det N
    VP -> V NP
##Lexikalische Regeln:
    Det -> 'the'
    Adj -> 'old'    
    N -> 'man' | 'boat' | 'old'
    V -> 'man'
""")


---
### 2.1 ShiftReduceParser im NLTK: ohne Backtracking implementiert

##### findet keinen Parse!

In [9]:
#NLTK-ShiftReduceParser (bottom-up-Parser): kein Backtracking! 
#bleibt bei Analyse NP NP ((the old man) (the boat)) stehen, findet keinen vollständigen Parse

parser = nltk.ShiftReduceParser(grammar,trace=2)

for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print(unicodelines=True)

Parsing 'the old man the boat'
    [ * the old man the boat]
  S [ 'the' * old man the boat]
  R [ Det * old man the boat]
  S [ Det 'old' * man the boat]
  R [ Det Adj * man the boat]
  S [ Det Adj 'man' * the boat]
  R [ Det Adj N * the boat]
  R [ NP * the boat]
  S [ NP 'the' * boat]
  R [ NP Det * boat]
  S [ NP Det 'boat' * ]
  R [ NP Det N * ]
  R [ NP NP * ]


### 2.2 RecursiveDescentParser im NLTK mit Backtracking implementiert

In [10]:
##NLTK-RecursiveDescentParser (top-down-Parser): Backtracking implementiert 
## Backtracking ermöglicht Reanalyse der Struktur

parser = nltk.RecursiveDescentParser(grammar,trace=3)

for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print(unicodelines=True)
    

Parsing 'the old man the boat'
Start:
    [ * S ]
Expand: S -> NP VP
    [ * NP VP ]
Expand: NP -> Det Adj N
    [ * Det Adj N VP ]
Expand: Det -> 'the'
    [ * 'the' Adj N VP ]
Match: 'the'
    [ 'the' * Adj N VP ]
Expand: Adj -> 'old'
    [ 'the' * 'old' N VP ]
Match: 'old'
    [ 'the' 'old' * N VP ]
Expand: N -> 'man'
    [ 'the' 'old' * 'man' VP ]
Match: 'man'
    [ 'the' 'old' 'man' * VP ]
Expand: VP -> V NP
    [ 'the' 'old' 'man' * V NP ]
Expand: V -> 'man'
    [ 'the' 'old' 'man' * 'man' NP ]
Backtrack: 'the' match failed
Expand: N -> 'boat'
    [ 'the' 'old' * 'boat' VP ]
Backtrack: 'man' match failed
Expand: N -> 'old'
    [ 'the' 'old' * 'old' VP ]
Backtrack: 'man' match failed
Expand: NP -> Det N
    [ * Det N VP ]
Expand: Det -> 'the'
    [ * 'the' N VP ]
Match: 'the'
    [ 'the' * N VP ]
Expand: N -> 'man'
    [ 'the' * 'man' VP ]
Backtrack: 'old' match failed
Expand: N -> 'boat'
    [ 'the' * 'boat' VP ]
Backtrack: 'old' match failed
Expand: N -> 'old'
    [ 'the' * 'old